In [141]:
from jina import Document, DocumentArray
import numpy as np
from jina.types.arrays.document import cosine_distance, euclidean_distance_squared

In [142]:
#%load_ext autoreload
#%autoreload 2

In [253]:
n_features = 100

def get_document_arrays():
    d1 = Document(embedding=np.array([0,0,0,0,0]))
    d2 = Document(embedding=np.array([1,0,0,0,0]))
    d3 = Document(embedding=np.array([1,1,1,1,0]))
    d4 = Document(embedding=np.array([1,2,2,1,0]))

    d1_m = Document(embedding=np.array([0,0.1,0,0,0]))
    d2_m = Document(embedding=np.array([1,0.1,0,0,0]))
    d3_m = Document(embedding=np.array([1,1.2,1,1,0]))
    d4_m = Document(embedding=np.array([1,2.2,2,1,0]))
    d5_m = Document(embedding=np.array([4,5.2,2,1,0]))

    D  = DocumentArray([d1,d2,d3,d4])
    D2 = DocumentArray([d1_m, d2_m, d3_m, d4_m, d5_m])
    return D, D2

In [254]:
D, D2 = get_document_arrays()

In [270]:
E = np.stack(D.get_attributes('embedding'))
E.argsort(axis=1)

array([[0, 1, 2, 3, 4],
       [1, 2, 3, 4, 0],
       [4, 0, 1, 2, 3],
       [4, 0, 3, 1, 2]])

#### _compute_dist Euclidean distance

In [256]:
X = np.stack(D.get_attributes('embedding'))
Y = np.stack(D2.get_attributes('embedding'))

dists = D._compute_dist(X, Y, 'euclidean')
idx, dist = D._get_sorted_top_k(dists, 2)
dist

array([[0.01, 1.01],
       [0.01, 1.01],
       [0.04, 2.44],
       [0.04, 1.64]])

In [257]:
idx, dist = D._get_sorted_top_k(dists,3)
dist

array([[0.01, 1.01, 4.44],
       [0.01, 1.01, 3.44],
       [0.04, 2.44, 2.81],
       [0.04, 1.64, 8.61]])

#### Computing matches Euclidean distance

In [263]:
D, D2 = get_document_arrays()
D.match(D2, metric='euclidean', limit=3)

In [264]:
print(f'query={D[2].embedding}')
for m in D[2].matches:
    print(m.embedding, 'dist=', m.scores['euclidean'].value)

query=[1 1 1 1 0]
[1.  1.2 1.  1.  0. ] dist= 0.03999999910593033
[1.  2.2 2.  1.  0. ] dist= 2.440000057220459
[1.  0.1 0.  0.  0. ] dist= 2.809999942779541


In [265]:
print(f'query={D[1].embedding}')
for m in D[1].matches:
    print(m.embedding, 'dist=', m.scores['euclidean'].value)

query=[1 0 0 0 0]
[1.  0.1 0.  0.  0. ] dist= 0.009999999776482582
[0.  0.1 0.  0.  0. ] dist= 1.0099999904632568
[1.  1.2 1.  1.  0. ] dist= 3.440000057220459


#### Computing matches Cosine similarity

In [268]:
D, D2 = get_document_arrays()
D.match(D2, metric='cosine', limit=3)

/Users/davidbuchaca/Documents/git_stuff/jina/jina/types/arrays/document.py:523: RuntimeWarning: invalid value encountered in true_divide
  )


In [269]:
print(f'query={D[2].embedding}')
for m in D[2].matches:
    print(m.embedding, 'dist=', m.scores['cosine'].value)

query=[1 1 1 1 0]
[1.  1.2 1.  1.  0. ] dist= 0.9966158866882324
[1.  2.2 2.  1.  0. ] dist= 0.9415579438209534
[4.  5.2 2.  1.  0. ] dist= 0.8800925612449646


In [275]:
cosine_distance(DE,DE)

array([[           nan,            nan,            nan,            nan],
       [           nan, 0.00000000e+00, 5.00000000e-01, 6.83772234e-01],
       [           nan, 5.00000000e-01, 0.00000000e+00, 5.13167019e-02],
       [           nan, 6.83772234e-01, 5.13167019e-02, 2.22044605e-16]])

In [276]:

def buch_cosine_distance(X, Y):
    dists = 1- np.dot(X, Y.T)/np.outer(np.linalg.norm(X, ord=2, axis=1),np.linalg.norm(Y, ord=2, axis=1))
    return dists

In [279]:
buch_cosine_distance(DE,DE)

<ipython-input-276-540327f3fa49>:2: RuntimeWarning: invalid value encountered in true_divide
  dists = 1- np.dot(X, Y.T)/np.outer(np.linalg.norm(X, ord=2, axis=1),np.linalg.norm(Y, ord=2, axis=1))


array([[           nan,            nan,            nan,            nan],
       [           nan, 0.00000000e+00, 5.00000000e-01, 6.83772234e-01],
       [           nan, 5.00000000e-01, 0.00000000e+00, 5.13167019e-02],
       [           nan, 6.83772234e-01, 5.13167019e-02, 2.22044605e-16]])

In [280]:
DE

array([[0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 1, 1, 1, 0],
       [1, 2, 2, 1, 0]])

In [235]:
DE = np.stack(D.get_attributes('embedding'))
D2E = np.stack(D2.get_attributes('embedding'))
cosine_distance(DE, D2E)

/Users/davidbuchaca/Documents/git_stuff/jina/jina/types/arrays/document.py:523: RuntimeWarning: invalid value encountered in true_divide
  )


array([[       nan,        nan,        nan,        nan,        nan],
       [1.        , 0.00496281, 0.525421  , 0.69627163, 0.42289014],
       [0.5       , 0.45272955, 0.0033841 , 0.05844205, 0.11990747],
       [0.36754447, 0.62240993, 0.03951964, 0.00110762, 0.11488392]])

In [151]:
DE = np.stack(D.get_attributes('embedding'))
D2E = np.stack(D2.get_attributes('embedding'))

euclidean_distance_squared(DE,D2E)

array([[1.000e-02, 1.084e+01],
       [1.010e+00, 9.840e+00],
       [3.810e+00, 2.440e+00],
       [9.610e+00, 4.000e-02]])

With metric='cosine':

```
array([[       nan,        nan],
       [1.        , 0.69627163],
       [0.5       , 0.05844205],
       [0.36754447, 0.00110762]])
```

In [152]:
#E = np.stack(darr.get_attributes('embedding'))
#print(E.shape)